# Système de Recommandation de Films

Ce notebook a pour objectif **de construire le dataset qui sera utilisé lors de la création de nos différents modèles de NLP.**

Le dataset initial comporte:


*   100 000 lignes
*   9 colonnes

Le dataset final comporte:


*   48297 lignes
*   3 colonnes


les colonnes pertinentes pour notre analyse:
- Labels (positif si rating > 5 et Négatif sinon)
- movie: le nom du film
- text (concattenation de review_summary et review_details)



In [ ]:
import pandas as pd
import os
from google.colab import drive

In [ ]:
drive.mount('/content/drive')
#os.chdir('path_to_your_data')
os.getcwd()

## 1. Exploration des Données et création du dataset

In [ ]:
#Importation des données
df = pd.read_json('sample.json')  #'part-06.json'
df.shape

(100000, 9)

In [ ]:
df.head()

,review_id,reviewer,movie,rating,review_summary,review_date,spoiler_tag,review_detail,helpful
0,rw1133942,OriginalMovieBuff21,Kill Bill: Vol. 2 (2004),8.0,Good follow up that answers all the questions,24 July 2005,0,"After seeing Tarantino's Kill Bill Vol: 1, I g...","[0, 1]"
1,rw1133943,sentra14,Journey to the Unknown (1968– ),NaN,Excellent series,24 July 2005,0,"I have the entire series on video, taped mostl...","[11, 11]"
2,rw1133946,GreenwheelFan2002,The Island (2005),9.0,"Not just about action, but about survival...",24 July 2005,0,Once again the critics prove themselves as mor...,"[2, 5]"
3,rw1133948,itsascreambaby,Win a Date with Tad Hamilton! (2004),3.0,Falls under the category: seen it a million ti...,24 July 2005,0,This IS a film that has been done too many tim...,"[2, 3]"
4,rw1133949,OriginalMovieBuff21,Saturday Night Live: The Best of Chris Farley ...,10.0,"Before Tommy Boy and Black Sheep, there was Sa...",24 July 2005,0,Chris Farley is one of my favorite comedians a...,"[4, 4]"


In [ ]:
df.columns

Index(['review_id', 'reviewer', 'movie', 'rating', 'review_summary',
       'review_date', 'spoiler_tag', 'review_detail', 'helpful'],
      dtype='object')

**Creation du champ review et label et équilibrage des données**

In [ ]:
def null_values_summary(df, percent=0):
    total = df.isnull().sum()
    pct = (total / len(df)) * 100
    missing_data = pd.concat([total, pct], axis=1, keys=['Total', 'Percent'])
    filtered_data = missing_data[(missing_data['Total'] != 0) & (missing_data['Percent'] >= percent)]
    return filtered_data.sort_values('Total', ascending=False)

In [ ]:
def preprocess_dataframe(df):
    """
    Preprocesses the DataFrame by handling missing ratings, creating a 'review' column,
    converting rating to integer, and creating a 'label' column.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The preprocessed DataFrame.
    """
    # Supprimer les lignes avec des ratings manquants
    df = df.dropna(subset=["rating"])

    # Créer la colonne 'review'
    df["review"] = df["review_summary"].fillna("") + " " + df["review_detail"].fillna("")

    # Convertir le rating en entier
    df["rating"] = df["rating"].astype(int)

    # Créer la colonne 'label'
    df["label"] = df["rating"].apply(lambda x: "positif" if x > 5 else "négatif")
    #df["label"] = df["rating"].apply(lambda x: "positif" if x > 5 else ("négatif" if x < 5 else "neutre"))

    return df

In [ ]:
# Valeurs manquantes
df["rating"] = pd.to_numeric(df["rating"], errors="coerce")
null_values_summary(df)

,Total,Percent
rating,12092,12.092


In [ ]:
df = preprocess_dataframe(df)
df.head()

<ipython-input-16-462b61c323bb>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["review"] = df["review_summary"].fillna("") + " " + df["review_detail"].fillna("")
<ipython-input-16-462b61c323bb>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["rating"] = df["rating"].astype(int)
<ipython-input-16-462b61c323bb>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,review_id,reviewer,movie,rating,review_summary,review_date,spoiler_tag,review_detail,helpful,review,label
0,rw1133942,OriginalMovieBuff21,Kill Bill: Vol. 2 (2004),8,Good follow up that answers all the questions,24 July 2005,0,"After seeing Tarantino's Kill Bill Vol: 1, I g...","[0, 1]",Good follow up that answers all the questions ...,positif
2,rw1133946,GreenwheelFan2002,The Island (2005),9,"Not just about action, but about survival...",24 July 2005,0,Once again the critics prove themselves as mor...,"[2, 5]","Not just about action, but about survival... O...",positif
3,rw1133948,itsascreambaby,Win a Date with Tad Hamilton! (2004),3,Falls under the category: seen it a million ti...,24 July 2005,0,This IS a film that has been done too many tim...,"[2, 3]",Falls under the category: seen it a million ti...,négatif
4,rw1133949,OriginalMovieBuff21,Saturday Night Live: The Best of Chris Farley ...,10,"Before Tommy Boy and Black Sheep, there was Sa...",24 July 2005,0,Chris Farley is one of my favorite comedians a...,"[4, 4]","Before Tommy Boy and Black Sheep, there was Sa...",positif
5,rw1133950,Aaron1375,Outlaw Star (1998– ),10,Great anime series soars through the stars.,24 July 2005,0,"I love this anime series, my only complaint is...","[11, 12]",Great anime series soars through the stars. I ...,positif


In [ ]:
# # Ratings null pour les predictions futures via le modèle créé
# invalid_ratings_df = df[df["rating"].isna()] #| (df["rating"] == "")
# invalid_ratings_10= invalid_ratings_df.head(10)
# invalid_ratings_10.to_csv("invalid_ratings.csv", index=False)
# invalid_ratings_10.shape

In [ ]:
df['label'].value_counts()

,count
label,
positif,64611
négatif,23297


In [ ]:
def balance_positive_negative_samples(df, n_samples=25000, random_state=42):
    """
    Balances the number of 'positif' and 'négatif' samples in the DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame with a 'label' column.
        n_samples (int): The number of 'positif' samples to sample.
        random_state (int): The random state for reproducibility.

    Returns:
        pd.DataFrame: The balanced DataFrame.
    """
    # Échantillonner les critiques positives
    df_positif = df[df['label'] == 'positif'].sample(n=n_samples, random_state=random_state)

    # Sélectionner toutes les critiques négatives
    df_negatif = df[df['label'] == 'négatif']
    #df_negatif = df[df['label'] == 'négatif'].sample(n=11000, random_state=random_state)

    # Concaténer les critiques positives échantillonnées et toutes les critiques négatives
    df_balanced = pd.concat([df_positif, df_negatif])

    # Mélanger les données pour éviter tout biais d'ordre
    df_balanced = df_balanced.sample(frac=1, random_state=random_state).reset_index(drop=True)

    return df_balanced

In [ ]:
# Utiliser la fonction pour obtenir le DataFrame équilibré
data = balance_positive_negative_samples(df)

# Afficher la distribution des labels dans le DataFrame équilibré
data['label'].value_counts()

,count
label,
positif,25000
négatif,23297


In [ ]:
# #Dataset for the database
# data.to_json("reviews_labeled.json", orient="records", force_ascii=False, indent=2)
data = data[["movie", "review", "label"]]
data = data.rename(columns={'review': 'text', 'label': 'labels'})
data.to_csv("reviews.csv", index=False)

In [ ]:
#data= pd.read_csv("reviews.csv")
data.head()

,movie,text,labels
0,Kiss Kiss Bang Bang (2005),Now 'This' is a Funny thrill ride of a movie t...,positif
1,Insider (1999),A real good miniseries I just saw this mini-se...,positif
2,Diamonds (1999),Fun Movie A movie doesn't need lots of hi-tech...,positif
3,Vivacious Lady (1938),good film but something is lacking in the chem...,positif
4,Philadelphia (1993),great drama movie Philadelphia is one of the b...,positif


In [ ]:
data['labels'].value_counts()

,count
labels,
positif,25000
négatif,23297
